This code divides the data into days with two hours overlap and does the wavelet.
You can use netCDF or csv data.

In [1]:
from __future__ import division
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib import colors
from matplotlib.collections import LineCollection
from datetime import datetime, timedelta
import pycwt as wavelet
from netCDF4 import Dataset

In [4]:
# function that calculates and returns the horizontal component of the magnetic field
def horizontal(df):
    Bn = df['dbn_nez'].to_numpy()
    Be = df['dbe_nez'].to_numpy()
    Bh = np.sqrt(Bn**2+Be**2)
    return Bh

In [5]:
# function that does the wavelet transform for given data
def wave(dat):
    dt = 60.0 # timestep 
    mother = wavelet.Morlet(6) 
    s0 = 2 * dt  # Starting scale, 2 * 60 s = 2 mins
    dj = 1 / 12  # Twelve sub-octaves per octaves
    J = 9 / dj  # Seven powers of two with dj sub-octaves

    # wavelet transform
    wave, scales, freqs, coi, fft, fftfreqs = wavelet.cwt(dat, dt, dj, s0, J,
                                                    mother)
    
    # inverse wavelet
    iwave = wavelet.icwt(wave, scales, dt, dj, mother)

    # ULF power
    power = (np.abs(wave)) ** 2
    
    power /= scales[:, None]

    return iwave, power, freqs

In [6]:
# takes the sum of powers in the pc5 range
def pc5power(freqs, power):
    # choose values in the pc5 frequency range
    a = np.where((2e-3 <= freqs) & (freqs <= 7e-3))
    low = a[0][0]
    up = a[0][-1]
    
    # sum the powers in the pc5 range
    powers = np.sum(power[low:up+1,:],axis=0)

    return powers

If you have netCDF data run the next two cells and skip the CSV data cell.
If you have CSV data skip the next two cells and run the csv data cell.

In [142]:
# read netCDF data
f = Dataset('filename.netcdf')

# get the needed variables
Bn = f.variables['dbn_nez'][:]
Be = f.variables['dbe_nez'][:]

yr = f.variables['time_yr'][:] #year
mo = f.variables['time_mo'][:] #months
day = f.variables['time_dy'][:] #days
hr = f.variables['time_hr'][:] #hours
mt = f.variables['time_mt'][:] #minutes
mlat = f.variables['mlat'][:] #magnetic latitudes
glon = f.variables['glon'][:] #geographic longitude
glat = f.variables['glat'][:] #geographic latitude
mlt = f.variables['mlt'][:]
stations = f.variables['id'][0][:]

lon = glon[0]
lat = glat[0]

# make dates into datetime object
date_UTC = [datetime(year=yr[i], month=mo[i], day=day[i],hour=hr[i],minute=mt[i]) for i in range(len(mo))]

# take stations between 60 and 70 magnetic latitudes
s6070 = np.where((mlat[0] >= 60) & (mlat[0] <70))

lons = lon[s6070]
lats = lat[s6070]

df_CDF = []

for i in s6070[0]:
    # make a dataframe of the relevant data
    data_df = {'dbn_nez': Bn[:,i], 'dbe_nez':Be[:,i], 'MLT':mlt[:,i]}
    # append the dataframe to a list
    df_CDF.append(pd.DataFrame(data=data_df, index=date_UTC).rename_axis(index='Date_UTC'))

In [143]:
# ULF power from netCDF data

# timestamps to divide data into days
time = df_CDF[0].index
start = time[0]
timerange = len(time)/(60*24)
end = start + timedelta(days=timerange)
indices = [start + timedelta(days=i) for i in range(int(timerange))]

dfs = []

for df in df_CDF:
    for date in indices:
        if date == start: # adds 2 hours of data to the end
            s = date
            e = date + timedelta(days=1, hours=2)

        elif date == end -timedelta(days=1): # adds two hours of data to the beginning
            s = date - timedelta(hours=2)
            e = end

        else: # adds two hours of data to both sides
            s = date -timedelta(hours=2)
            e = date + timedelta(days=1, hours=2)

        # take the day in question (plus the overlaps)
        df1 = df[(df.index >= s) & (df.index < e)]
        part = df1.index # times
        
        # get the horizontal component and make a series
        B = horizontal(df1)
        ser = pd.Series(data=B, index=part)

        # maximum count of consecutive nans
        nans = ser.isnull().astype(int).groupby(ser.notnull().astype(int).cumsum()).sum().max()

        # if there is a datagap larger than 40 minutes or the data starts with zero, do not include 
        if nans > 40 or ser.isna()[0] == True:
            continue

        # interpolate for possible nans
        ser = ser.interpolate()
        ser.resample('1s').fillna('bfill')
        B = ser.to_numpy()
        
        # get the wavepower
        iwave, power, freqs = wave(B)

        # pc5 power
        pc5 = pc5power(freqs, power)
        dat = {'pc5' : pc5, 'MLT' : df1['MLT'].to_numpy()}
        df2 = pd.DataFrame(data=dat, index=part)

        # remove overlaps and save to a list
        dfs.append(df2[(df2.index>= date) & (df2.index < date+ timedelta(days=1))])

Run the cell below if you are using csv data
If you are using netCDF data skip this next cell

In [319]:
# read csv data
df = pd.read_csv('filename.csv')

# convert dates into datetime 
format='%Y-%m-%dT%H:%M:%S'
df['Date_UTC'] = pd.to_datetime(df['Date_UTC'], format=format)

# get the time of the whole data
times = df.groupby('Date_UTC').mean()

# get names of the stations
names = df.groupby('IAGA').count().index.to_numpy()

# timestamps to divide data into days
start = times.index[0]
timerange = len(times)/(60*24)
end = start + timedelta(days=timerange)
indices = [start + timedelta(days=i) for i in range(int(timerange))]

dfs = []

# go over dates
for date in indices:
    if date == start: # adds 2 hours of data to the end
        s = date
        e = date + timedelta(days=1, hours=2)

    elif date == end -timedelta(days=1): # adds two hours of data to the beginning
        s = date - timedelta(hours=2)
        e = end

    else: # adds two hours of data to both sides
        s = date -timedelta(hours=2)
        e = date + timedelta(days=1, hours=2)

    # go through all stations 
    for name in names:
        # take the station and day in question
        a = df.groupby('IAGA').get_group(name)
        df1 = a[(a['Date_UTC'] >= s) & (a['Date_UTC'] < e)]
        time = df1['Date_UTC']

        B = horizontal(df1)
        ser = pd.Series(data=B, index=df1['Date_UTC'])

        # maximum count of consecutive nans
        nans = ser.isnull().astype(int).groupby(ser.notnull().astype(int).cumsum()).sum().max()
        
        # if there is a datagap larger than 40 minutes or the data starts with zero, do not include 
        if nans > 40 or ser.isna()[0] == True:
            continue

        # interpolate possible nan values
        ser = ser.interpolate()
        ser.resample('1s').fillna('bfill')
        B = ser.to_numpy()
        
        # get the wavepower
        iwave, power, freqs = wave(B, 60.0)

        # pc5 power
        pc5 = pc5power(freqs, power)
        dat = {'pc5' : pc5, 'MLT' : df1['MLT'].to_numpy()}
        df2 = pd.DataFrame(data=dat, index=time)

        # remove overlaps and save to a list
        dfs.append(df2[(df2.index >= date) & (df2.index < date + timedelta(days=1))])

Next cells are the same for both kinds of data.

In [144]:
# merge all dataframes together and calculate the mean pc5 power
con = pd.concat(dfs)
global_ULF = con.groupby('Date_UTC')['pc5'].mean()
time = global_ULF.index

# get hourly mean of pc5 power
glbl_ULF = pd.Series(global_ULF.values, name='ULF power', index=time)
hmean = glbl_ULF.resample('1H').mean()

In [145]:
# divide data into different MLT sectors

idx = pd.date_range(min(time), max(time), freq='T')

# ULF power in the night sector
MLT213 = con[((con['MLT']>= 21) & (con['MLT'] <= 24)) | ((con['MLT']>= 0) & (con['MLT'] < 3))]
night = MLT213.groupby('Date_UTC')['pc5'].mean()
nightstat = MLT213.groupby('Date_UTC').size()

# if dates are missing, add zeros to them
if len(night) != len(time):
    night = night.reindex(idx, fill_value=0)
    nightstat = nightstat.reindex(idx, fill_value=0)

# ULF power in the dawn sector
MLT39 = con[(con['MLT'] >= 3) & (con['MLT'] < 9)]
dawn = MLT39.groupby('Date_UTC')['pc5'].mean()
dawnstat = MLT39.groupby('Date_UTC').size()

# if dates are missing, add zeros to them
if len(dawn) != len(time):
    dawn = dawn.reindex(idx, fill_value=0)
    dawnstat = dawnstat.reindex(idx, fill_value=0)

# ULF power in the day sector
MLT915 = con[(con['MLT'] >= 9) & (con['MLT'] < 15)]
day = MLT915.groupby('Date_UTC')['pc5'].mean()
daystat = MLT915.groupby('Date_UTC').size()

# if dates are missing, add zeros to them
if len(day) != len(time):
    day = day.reindex(idx, fill_value=0)
    daystat = daystat.reindex(idx, fill_value=0)

# ULF power in the dusk sector
MLT1521 = con[(con['MLT'] >= 15) & (con['MLT'] < 21)]
dusk = MLT1521.groupby('Date_UTC')['pc5'].mean()
duskstat = MLT1521.groupby('Date_UTC').size()

# if dates are missing, add zeros to them
if len(dusk) != len(time):
    dusk = dusk.reindex(idx, fill_value=0)
    duskstat = duskstat.reindex(idx, fill_value=0)

# add together
datas = {'D':night.index.day, 'H':night.index.hour, 'MIN' : night.index.minute, 'Tdawn':dawn.to_numpy(), 'Tday':day.to_numpy(), 'Tdusk':dusk.to_numpy(), 'Tnight':night.to_numpy(),'Tmean':global_ULF.to_numpy()}
d = pd.DataFrame(data=datas)

In [323]:
# write to daily files
for i in indices:
    dayw = d[(d['D']==i.day)]
    path = '/home/live/ULF_index/'+i.strftime('%Y')+'/'+i.strftime('%m')+'/'+i.strftime('%d')+'.asc'
    with open(path, "w") as f:
        df_string = dayw.to_string(index=False)
        f.write(df_string)

Next cells are for plotting

In [3]:
# GET ULF INDICES
import read_virbo_ULF as VULF

#start
year_s = 2001
month_s = 1
day_s = 1
start_time = str(year_s)+'-'+str(month_s)+'-'+str(day_s)

#end
year_e = 2001
month_e = 12
day_e = 31
end_time = str(year_e)+'-'+str(month_e)+'-'+str(day_e)

startTime = datetime(year_s, month_s, day_s)
stopTime = datetime(year_e, month_e, day_e)

timeTgr, Tgr, Tgeo , Tgr_detrend, Tgeo_detrend= VULF.read_ULF_asc_data(startTime, stopTime)

dates = []
for i in timeTgr:
    dates.append(datetime.fromtimestamp(i))

In [132]:
# function to plot a multicolor line
def multicolor(y, s, ax):
    # plot grey lines
    ax.axhline(1e-3, c='grey', alpha=0.4, ls='--')
    ax.axhline(1e-1, c='grey', alpha=0.4, ls='--')
    ax.axhline(1e1, c='grey', alpha=0.4, ls='--')
    ax.axhline(1e3, c='grey', alpha=0.4, ls='--')

    xval = mdates.date2num(global_ULF.index)
    points = np.array([xval, y]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)
    cmap = colors.LinearSegmentedColormap.from_list("", ["pink","hotpink","deeppink","mediumvioletred"])
    s = np.ma.masked_where(s<1,s)
    cmap.set_bad(color='white')
    lc = LineCollection(segments, cmap=cmap)
    lc.set_array(s)
    lc.set_linewidth(3)

    line = ax.add_collection(lc)
    
    return line

In [ ]:
# plot ULF data
fontsize=18

fig, ax = plt.subplots(7, 1, figsize=(21,25), sharex='all')
fig.set(facecolor='white')
fig.suptitle('Magnetic latitude: 60–70, horizontal B', fontsize=20)

l = multicolor(night, nightstat, ax[0])
ax[0].set_ylabel('Night', fontsize=fontsize)
ax[0].set_yscale('log')

multicolor(dawn, dawnstat, ax[1])
ax[1].set_ylabel('Dawn', fontsize=fontsize)
ax[1].set_yscale('log')

multicolor(day, daystat, ax[2])
ax[2].set_ylabel('Day', fontsize=fontsize)
ax[2].set_yscale('log')

multicolor(dusk, duskstat, ax[3])
ax[3].set_ylabel('Dusk', fontsize=fontsize)
ax[3].set_yscale('log')

ax[4].plot(time, global_ULF, color='darkviolet')
ax[4].set_ylabel('mean pc5 (min)', fontsize=fontsize)

ax[5].plot(hmean.index, hmean.values, color='darkviolet')
ax[5].set_ylabel('mean pc5 (h)', fontsize=fontsize)

ax[6].plot(dates, Tgr, color='darkviolet')
ax[6].set_ylabel(r'$\rm{T_{gr}}$', fontsize=fontsize)
ax[6].set_xlim([time.min(), time.max()])
#ax[6].set_yscale('log')
ax[6].tick_params(axis='x', labelrotation = 40)
ax[6].xaxis.set_major_formatter(mdates.DateFormatter('%y-%m-%d %H:%M:%S'))

plt.rc('xtick', labelsize=16)    
plt.rc('ytick', labelsize=15) 

cax = plt.axes([0.92, 0.57, 0.015, 0.3])
fig.colorbar(l,cax=cax)

plt.subplots_adjust(hspace=0.0)
fig.align_ylabels()
fig.subplots_adjust(top=0.95)
plt.show()